In [1]:
import os

import numpy as np
from tqdm import tqdm
from scipy.stats import truncnorm
import sys
import warnings

sys.path.append('../')
sys.path.append('../../')
from DataBot.data_bot import BotMLP
from Boards.Speed_leedo_2p_v6 import FullBoard
from Intelligent_bot.intelligent_bot import Bot
import gc

env = FullBoard()
agent0 = BotMLP()
bot0 = Bot()

wins = []

agent0.load_model("../../DataBot/models/mlp_bot_models/model_{}.hdf5".format(4))

warnings.filterwarnings('ignore')

for ep in tqdm(range(0, 10000), ascii=True, unit="e"):
    s, _, game_over, player_turn = env.reset()
    episode_reward = [0.0, 0.0]

    time = 0
    while time < 360 or player_turn == 0:

        time += truncnorm.rvs(-2.5, 2.5, loc=2.5, scale=1)
        player_turn_temp = env.get_player_turn()
        env.roll_dice()[0]
        player_turn = env.get_player_turn()

        if player_turn == player_turn_temp:

            action_list = env.get_next_states(player_turn)

            if action_list:
                if player_turn == 0:
                    s_t = env.return_state()
                    action = agent0.act(np.array(s_t).astype('float32'), action_list)
                else:
                    s_t = env.return_state()
                    action = bot0.act(state=s_t, p=player_turn)

                time += truncnorm.rvs(-2.5, 2.5, loc=2.5, scale=1)
                s_, reward, game_over, player_turn_temp = env.make_step(action)

                episode_reward[player_turn] += reward[player_turn]
       
    if episode_reward[0] > episode_reward[1]:
        wins.append(1)
    else:
        wins.append(0)

    if ep % 1000 == 0 and ep != 0:
        print(np.sum(wins))

2022-10-21 06:10:35.693076: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-21 06:10:35.705915: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-21 06:10:35.706451: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-21 06:10:35.723524: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [ ]:
np.sum(wins)

3890